# Silver — CALLS

Este notebook **atualiza a tabela Silver de CALLS** consumindo **apenas as mudanças** da Bronze por **Change Data Feed (CDF)** e aplicando **MERGE**.
O processo usa uma **tabela de controle** que guarda a última versão (`_commit_version`) aplicada.

**Fluxo**
1. Lê o **marcador** de versão na tabela de controle.
2. Lê o **CDF** da Bronze desde esse marcador.
3. Faz **parse mínimo** do `body` (JSON) com `StructType`, renomeia colunas e deriva campos de data.
4. **Garante** que a Silver exista com o **mesmo schema**.
5. Aplica **MERGE** por chave de negócio.
6. Atualiza o **marcador** com a maior versão aplicada.


## Parâmetros


In [0]:
# ===================== PARÂMETROS =====================
CATALOG        = "prd"
BRONZE_SCHEMA  = "b_dm_callcenter"
SILVER_SCHEMA  = "s_dm_callcenter"
BRONZE_TABLE   = "calls_once"
SILVER_TABLE   = "tabe_calls"

# FQNs
BRONZE_FQN = f"{CATALOG}.{BRONZE_SCHEMA}.{BRONZE_TABLE}"
SILVER_FQN = f"{CATALOG}.{SILVER_SCHEMA}.{SILVER_TABLE}"

# Tabela de controle
CTRL_TBL_FQN = f"{CATALOG}.{SILVER_SCHEMA}.__last_processed_version"

print("Bronze:", BRONZE_FQN)
print("Silver:", SILVER_FQN)
print("Ctrl  :", CTRL_TBL_FQN)

## Imports

In [0]:

from pyspark.sql import functions as F, types as T, Window as W



## Helpers — Tabela de controle
Lê/atualiza o marcador de versão.


In [0]:
def get_last_version(src_fqn: str) -> int:
    """Retorna a última versão (_commit_version) processada para a fonte."""
    row = (spark.table(CTRL_TBL_FQN)
             .filter(F.col("source_table")==src_fqn)
             .select("last_version")
             .limit(1)
             .collect())
    return int(row[0][0]) if row else 0

def set_last_version(src_fqn: str, new_version: int):
    """Atualiza o marcador com a última versão aplicada."""
    spark.sql(f"""
        MERGE INTO {CTRL_TBL_FQN} t
        USING (SELECT '{src_fqn}' AS source_table, {int(new_version)} AS last_version, current_timestamp() AS last_run_at) s
        ON t.source_table = s.source_table
        WHEN MATCHED THEN UPDATE SET *
        WHEN NOT MATCHED THEN INSERT *
    """)


## Leitura do CDF da Bronze
Lê apenas mudanças a partir do `startingVersion`.


In [0]:
def read_bronze_cdf(src_fqn: str, starting_version: int):
    return (spark.read
            .format("delta")
            .option("readChangeFeed", "true")
            .option("startingVersion", starting_version)
            .table(src_fqn))


## Transformação

In [ ]:
def transform(df_raw):
    # Schema do JSON
    schema = T.StructType([
        T.StructField("id_chamada", T.StringType(), True),
        T.StructField("id_atendimento", T.StringType(), True),
        T.StructField("id_cliente", T.StringType(), True),
        T.StructField("id_assistente", T.IntegerType(), True),
        T.StructField("data_hora_inicio", T.TimestampType(), True),
        T.StructField("data_hora_fim", T.TimestampType(), True),
        T.StructField("area_atendimento", T.StringType(), True),
    ])

    rename = {
        "id_chamada":          "ID_CHAM",
        "id_atendimento":      "ID_ATEN",
        "id_cliente":          "ID_CLIE",
        "id_assistente":       "ID_ASST",
        "data_hora_inicio":    "DH_INIC",
        "data_hora_fim":       "DH_FIM",
        "area_atendimento":    "DS_AREA_ATEN"
    }

    df = (df_raw
        .withColumn("body", F.from_json(F.col("body"), schema))
        .filter(F.col("body").isNotNull())
        .select("body.*")
    )

    # Renomeia as colunas
    df = df.select([F.col(c).alias(rename.get(c, c)) for c in df.columns])

    # Colunas de DATA do evento e carga
    df = df \
        .withColumn('CD_PERI', F.date_format(F.col('DH_INIC'), 'yyyyMM').cast('int')) \
        .withColumn('DT_INIC', F.to_date('DH_INIC')) \
        .withColumn('DT_FIM',  F.to_date('DH_FIM')) \
        .withColumn('DH_REFE_CRGA', F.current_timestamp())

    return df

## Garantir criação da Silver com o **mesmo schema** do staged
Cria a tabela **vazia** com o schema do `staged_df` se ela ainda não existir.

In [0]:
def ensure_table_exists(silver_fqn: str, staged_df):
    parts = silver_fqn.split(".")
    if len(parts) != 3:
        raise ValueError(f"FQN inválido: {silver_fqn}. Use catalog.schema.table")
    
    # Checa existência via catálogo
    if not spark.catalog.tableExists(silver_fqn):
        empty = spark.createDataFrame([], staged_df.schema)
        (empty.write
              .format("delta")
              .mode("overwrite")
              .saveAsTable(silver_fqn))
        spark.sql(f"ALTER TABLE {silver_fqn} CLUSTER BY (CD_PERI,DT_INIC,ID_CHAM)")

## MERGE
- **MATCHED** → `UPDATE SET *`
- **NOT MATCHED** → `INSERT *`

In [0]:
def merge(silver_fqn: str, staged_df, keys):
    ensure_table_exists(silver_fqn, staged_df)
    staged_df.createOrReplaceTempView("_stg_silver_calls")
    on_clause = " AND ".join([f"S.{k} = C.{k}" for k in keys])
    spark.sql(f"""
        MERGE INTO {silver_fqn} AS S
        USING _stg_silver_calls AS C
        ON {on_clause}
        WHEN MATCHED THEN UPDATE SET *
        WHEN NOT MATCHED THEN INSERT *
    """)



## ▶️ Execução
1) Lê marcador → 2) Lê CDF → 3) Filtra inserts → 4) Transforma → 5) MERGE → 6) Atualiza marcador.


In [0]:

# 1) marcador
last_v = get_last_version(BRONZE_FQN)
print(f"Última versão processada (CDF) para {BRONZE_FQN}: {last_v}")

# 2) CDF
df_bronze = read_bronze_cdf(BRONZE_FQN, starting_version=last_v)

# 3) Capturando as mudanças
df_cdf = df_bronze.filter(F.col("_change_type").isin("insert"))

# 4) TRANSFORMAÇÃO
df_staged = transform(df_cdf)

# 5) Aplicando marcações extras
lead_asst = F.lead('ID_ASST',1).over(W.partitionBy('ID_CHAM').orderBy(F.asc(F.col('DH_INIC'))))
lead_area = F.lead('DS_AREA_ATEN',1).over(W.partitionBy('ID_CHAM').orderBy(F.asc(F.col('DH_INIC'))))

df_traf = (
    df_staged
    .withColumn('IN_TRAF', F.when(lead_asst.isNotNull(), F.lit(1)).otherwise(F.lit(0)))
    .withColumn('IN_TRAF_INDV', F.when((F.col('IN_TRAF') == 1) & (F.col('DS_AREA_ATEN') == lead_area), F.lit(1)).otherwise(F.lit(0)))
)

# 6) MERGE
merge(SILVER_FQN, df_traf, keys=["ID_CHAM", "ID_ATEN"])

# 7) atualizar marcador com a MAIOR versão
max_version = df_cdf.agg(F.max("_commit_version")).first()[0]

if max_version:
    if max_version > last_v:
        set_last_version(BRONZE_FQN, int(max_version))
        print(f"Marcador atualizado → last_version = {int(max_version)}")
    else:
        print("Nenhuma mudança nova no CDF (nada a atualizar).")
else:
    print('Marcador vazio')
